In [1]:
from bs4 import BeautifulSoup
import datetime
from io import StringIO
import os
import pandas as pd
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

DATA_DIR = 'data'
artists = []
try:
    os.remove('data/Timeline_of_canon_media')
except:
    pass

async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.webkit.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

async def scrape_charts():
    url = "https://starwars.fandom.com/wiki/Timeline_of_canon_media"
    save_path = os.path.join(DATA_DIR, url.split('/')[-1])
    if not os.path.exists(save_path):
        html = await get_html(url, '.mw-parser-output')
        with open(save_path, 'w+') as f:
            f.write(html)

await scrape_charts()
table = os.listdir(DATA_DIR)[0]
table = os.path.join(DATA_DIR, table)

with open(table) as f:
        html = f.read()
soup = BeautifulSoup(html)

df = pd.read_html(StringIO(str(soup)))[2]
df = df.dropna(subset=['Released'])

available = df[(df['Unnamed: 1'] == 'F') | (df['Unnamed: 1'] == 'TV') | (df['Unnamed: 1'] == 'P')]
available.reset_index(drop=True)

current_date = datetime.datetime.now().strftime('%Y%m%d')
watch = available.sample(n=1, random_state=int(current_date))
watch

Timeline of canon media | Wookieepedia | Fandom


,Year,Unnamed: 1,Title,Released
547,20 BBY[6],TV,"Star Wars: The Clone Wars — ""Plan of Dissent""",2011-11-11
